# Deploy Scikit-learn Models to Amazon SageMaker with the SageMaker Python SDK using Script mode
> The aim of this notebook is to demonstrate how to train and deploy a scikit-learn model in Amazon SageMaker using script mode.

- toc: true 
- badges: true
- comments: true
- categories: [aws, ml, sagemaker]
- keyword: [aws, ml, sagemaker, sklearn, scikit-learn, python]
- image: images/copied_from_nb/images/2022-07-07-sagemaker-script-mode.jpeg

![](images/2022-07-07-sagemaker-script-mode.jpeg)

# Introduction
You may have trained a model with your favorite ML framework, and now you are asked to move your code to Amazon SageMaker. The good news is that SageMaker's fully managed training works well with many popular ML frameworks, including `scikit-learn.` In addition, SageMaker provides its prebuilt container for the scikit-learn framework, enabling us to port our scripts to SageMaker and benefit from its training and deployment capabilities. SageMaker's scikit-learn container is an open source library for making the scikit-learn framework run on the Amazon SageMaker platform. You can read more about sklearn container features from its GitHub page [SageMaker Scikit-learn Container](https://github.com/aws/sagemaker-scikit-learn-container).

Amazon SageMaker also provides open source Python SDK to train and deploy models on SageMaker. SageMaker SDK provides several high-level abstractions (classes), including:
* `Session` Provides a collection of methods for working with SageMaker resources 
* `Estimators` Encapsulate training on SageMaker
* `Predictors` Provide real-time inference and transformation using Python data types against a SageMaker endpoint
You can read more on SageMaker Python SDK from its official site [Amazon SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/overview.html)

This approach of using a custom training script with SageMaker's prebuilt container is commonly called **Script Mode**. To train a scikit-learn model by using the SageMaker Python SDK involves three steps:

1. **Prepare a training script**. The training script is similar to any other scikit-learn training script that you might use outside of SageMaker
2. **Create an Estimator object from class `sagemaker.sklearn.SKLearn`**. Scikit-learn estimator class handles end-to-end training and deployment of custom scikit-learn code. It will execute a scikit-learn script within a SageMaker Training Job. The managed scikit-learn environment is an Amazon-built Docker container that runs functions defined in the supplied `entry_point` Python script. 
3. **Call the Estimator's `fit` method on training data**. Training is started by calling `fit()` on this Estimator. After training is complete, calling `deploy()` creates a hosted SageMaker endpoint and returns a `SKLearnPredictor` instance that can be used to perform inference against the hosted model. We will discuss the `SKLearn` Estimator in more detail later in this post.

To read more about using scikit-learn with the SageMaker Python SDK, you may refer to the official documentation [using Scikit-learn with the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html). The official documentation is valuable, and I would highly recommend checking it and keeping it as a reference.

In this post we will built a scikit-learn [RandomForrestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) on [iris public dataset](https://archive.ics.uci.edu/ml/datasets/iris). There is a similar example in SageMaker documentation. [Train a SKLearn Model using Script Mode](https://sagemaker-examples.readthedocs.io/en/latest/sagemaker-script-mode/sklearn/sklearn_byom_outputs.html). Still, it does not discuss many important elements of a scikit-learn container and its environment. This post will cover all the stages of training a scikit-learn model with script mode. I also noted that the example in the documentation uses `RandomForrestRegressor` on a classification problem which I believe is a mistake.

We have much to cover and learn, so let's start.

# Environment
notebook environment.

# Prepare training and test data
We will use **Iris flower dataset**. It includes three iris species (Iris setosa, Iris virginica, and Iris versicolor) with 50 samples each and some properties about each flower. You can read more about this dataset at [Iris flower data set](https://en.wikipedia.org/wiki/Iris_flower_data_set)
1. sepal length in cm
2. sepal width in cm
3. petal length in cm
4. petal width in cm
5. class: Iris Setosa, Iris Versicolour, Iris Virginica

## Download and preprocess data

In [1]:
##
# download dataset
import boto3
import pandas as pd
import numpy as np

s3 = boto3.client("s3")
s3.download_file(
    f"sagemaker-sample-files", "datasets/tabular/iris/iris.data", "iris.data"
)

df = pd.read_csv(
    "iris.data",
    header=None,
    names=["sepal_len", "sepal_wid", "petal_len", "petal_wid", "class"],
)
df.head()

,sepal_len,sepal_wid,petal_len,petal_wid,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [2]:
##
# Convert the three classes from strings to integers in {0,1,2}
df["class_cat"] = df["class"].astype("category").cat.codes
categories_map = dict(enumerate(df["class"].astype("category").cat.categories))
print(categories_map)
df.head()

{0: 'Iris-setosa', 1: 'Iris-versicolor', 2: 'Iris-virginica'}


,sepal_len,sepal_wid,petal_len,petal_wid,class,class_cat
0,5.1,3.5,1.4,0.2,Iris-setosa,0
1,4.9,3.0,1.4,0.2,Iris-setosa,0
2,4.7,3.2,1.3,0.2,Iris-setosa,0
3,4.6,3.1,1.5,0.2,Iris-setosa,0
4,5.0,3.6,1.4,0.2,Iris-setosa,0


## Prepare and store train and test sets as CSV files

In [70]:
##
# split the data into train and test set
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)

print(f"train.shape: {train.shape}")
print(f"test.shape: {test.shape}")

train.shape: (120, 6)
test.shape: (30, 6)


We have our dataset ready. Let's define a local directory where we keep all the files and artifacts related to this post. I will refer to this directory as 'workspace'.

In [4]:
##
# `local_path` will be the root directory for this post.
local_path = "./datasets/2022-07-07-sagemaker-script-mode"

We have train and test sets ready. Let's create two more directories in our workspace and store our data in them.

In [71]:
from pathlib import Path

# local paths
local_train_path = f"{local_path}/train"
local_test_path = f"{local_path}/test"

# create local directories
Path(local_train_path).mkdir(parents=True, exist_ok=True)
Path(local_test_path).mkdir(parents=True, exist_ok=True)

print(f"local_train_path: {local_train_path}")
print(f"local_test_path: {local_test_path}")

# local file names
local_train_file = local_train_path + "/train.csv"
local_test_file = local_test_path + "/test.csv"

# write train and test CSV files
train.to_csv(local_train_file, index=False)
test.to_csv(local_test_file, index=False)

print(f"local_train_file: {local_train_file}")
print(f"local_test_file: {local_test_file}")

local_train_path: ./datasets/2022-07-07-sagemaker-script-mode/train
local_test_path: ./datasets/2022-07-07-sagemaker-script-mode/test
local_train_file: ./datasets/2022-07-07-sagemaker-script-mode/train/train.csv
local_test_file: ./datasets/2022-07-07-sagemaker-script-mode/test/test.csv


## Create SageMaker session

In [ ]:
import sagemaker

session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = session.default_bucket()
region = session.boto_region_name

print(f"sagemaker.__version__: {sagemaker.__version__}")
print(f"Session: {session}")
print(f"Role: {role}")
print(f"Bucket: {bucket}")
print(f"Region: {region}")

What we have done is
* imported the SageMaker Python SDK into our runtime
* get a session to work with SageMaker API and other AWS services
* get the execution role associated with the user profile. It is the same profile that is available to the user to work from console UI and has `AmazonSageMakerFullAccess` policy attached to it.
* create or get a default bucket to use and return its name. Default bucket name has the format `sagemaker-{region}-{account_id}`. If it doesn't exist then our session will automatically create it. You may also use any other bucket in its place given that you have enough permission for reading and writing.
* get the region name attached to our session

Next, we will use this session to upload data to our default bucket. 

## Upload data to Amazon S3 bucket

In [ ]:
##
# You may choose any other prefix for your bucket.
# All the data related to this post will be under this prefix.
bucket_prefix = "2022-07-07-sagemaker-script-mode"

Now upload the data. In the output, we will get the complete path (S3 URI) for our uploaded data.

In [72]:
s3_train_uri = session.upload_data(local_train_file, key_prefix=bucket_prefix + "/data")
s3_test_uri = session.upload_data(local_test_file, key_prefix=bucket_prefix + "/data")

print(f"s3_train_uri: {s3_train_uri}")
print(f"s3_test_uri: {s3_test_uri}")

s3_train_uri: s3://sagemaker-us-east-1-801598032724/sklearn-iris/data/train.csv
s3_test_uri: s3://sagemaker-us-east-1-801598032724/sklearn-iris/data/test.csv


At this point our data preparation step is complete. Training and test CSV files are available both on the local system, and in our default Amazon S3 bucket.

# Prepare SageMaker local environment
The Amazon SageMaker training environment is managed, but SageMaker Python SDK also supports **local mode**, allowing you to train and deploy models to your local environment. This is a great way to test training scripts before running them in SageMaker's managed training or hosting environment.

## How SageMaker managed environment works?
When you send a request to SageMaker API (fit or deploy call)
* it spins up new instances with the provided specification
* loads the algorithm container
* pulls the data from S3
* runs the training code
* store the results and trained model artifacts to S3
* terminates the new instances

All this happens behind the scenes with a single line of code and is a huge advantage. Spinning up new hardware every time can be good for repeatability and security, but it can add some friction while testing and debugging our code. We can test our code on a small dataset in our local environment with SageMaker local mode and then switch seamlessly to SageMaker managed environment by changing a single line of code.

## Steps to prepare Amazon SageMaker local environment
Install the following pre-requisites if you want to set up Amazon SageMaker on your local system.
1. Install required Python packages:
    ```
    pip install boto3 sagemaker pandas scikit-learn
    pip install 'sagemaker[local]'
    ```
2. Docker Desktop installed and running on your computer:
    ```
    docker ps
    ```
3. You should have AWS credentials configured on your local machine to be able to pull the docker image from ECR.

### Instructions for SageMaker notebook instances
You can also set up SageMaker's local environment in SageMaker notebook instances. Required Python packages and Docker service is already there. You only need to upgrade the `sagemaker[local]` Python package.

In [74]:
#collapse_output
# this is required for SageMaker notebook instances
!pip install 'sagemaker[local]' --upgrade

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


#### Instructions for SageMaker Studio environment
Note that SageMaker local mode will not work in SageMaker Studio environment as it does not have docker service installed on instances.

## Create SageMaker local session

In [76]:
from sagemaker.local import LocalSession

session_local = LocalSession()
session_local

In [77]:
##
# configure local session
session_local.config = {"local": {"local_code": True}}

# Prepare SageMaker training script

We will call our training script `train_and_serve.py` and place it in our workspace under the `/src` folder. Then, we will start with a simple `Hello World` message code. After that, we will update and complete our training script as we learn more about the SageMaker `scikit-learn` container environment.

In [78]:
script_file_name = "train_and_serve.py"
script_path = f"{local_path}/src"
script_file = script_path + "/" + script_file_name

print(f"script_file_name: {script_file_name}")
print(f"script_path: {script_path}")
print(f"script_file: {script_file}")

script_file_name: train_and_serve.py
script_path: ./datasets/2022-07-07-sagemaker-script-mode/src
script_file: ./datasets/2022-07-07-sagemaker-script-mode/src/train_and_serve.py


In [79]:
# make sure that the directory exists
Path(script_path).mkdir(parents=True, exist_ok=True)

Now the training script.

In [14]:
%%writefile $script_file

if __name__ == "__main__":
    print("*** Hello from the SageMaker script mode***")

Overwriting ./datasets/2022-07-07-sagemaker-script-mode/src/train_and_serve.py


# Prepare SageMaker SKLearn estimator

To use our `train_and_serve.py` script with SageMaker SKLearn estimator, we need to provide the following required items
* **`entry_point (str)`** Path (absolute or relative) to the Python source file, which should be executed as the entry point to training. If source_dir is specified, then entry_point must point to a file located at the root of source_dir.
* **`framework_version (str)`** Scikit-learn version you want to use for executing your model training code.
* **`role (str)`** An AWS IAM role (either name or full ARN)
* **`instance_type (str)`** Type of instance to use for training. For local mode use string **`local`**
* **`instance_count (int)`** Number of instances to use for training. Since we will train in the local environment and have a single instance, we will use '1' here.

You can read more about the SKLearn Estimator class from the official documentation [Scikit Learn Estimator](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html)

Let's find the SKLearn framework version.

In [15]:
import sklearn

print(sklearn.__version__)

1.0.1


Note that version number `1.0.1` has to be provided to the SKLearn estimator class as **`1.0-1`**. Otherwise, you will get the following error message.
```
ValueError: Unsupported sklearn version: 1.0.1. You may need to upgrade your SDK version (pip install -U sagemaker) for newer sklearn versions. Supported sklearn version(s): 0.20.0, 0.23-1, 1.0-1.
```

Now let us create the SageMaker SKLearn estimator object and pass our training script to it.

In [16]:
#collapse-output
from sagemaker.sklearn import SKLearn

sk_estimator = SKLearn(
    entry_point=script_file,
    role=role,
    instance_count=1,
    instance_type="local",
    framework_version="1.0-1",
)

sk_estimator.fit()

Creating uws5uf81g1-algo-1-sboaf ... 
Creating uws5uf81g1-algo-1-sboaf ... done
Attaching to uws5uf81g1-algo-1-sboaf
uws5uf81g1-algo-1-sboaf | 2022-07-13 13:05:47,717 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
uws5uf81g1-algo-1-sboaf | 2022-07-13 13:05:47,722 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
uws5uf81g1-algo-1-sboaf | 2022-07-13 13:05:47,731 sagemaker_sklearn_container.training INFO     Invoking user training script.
uws5uf81g1-algo-1-sboaf | 2022-07-13 13:05:47,886 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
uws5uf81g1-algo-1-sboaf | 2022-07-13 13:05:47,899 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
uws5uf81g1-algo-1-sboaf | 2022-07-13 13:05:47,912 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
uws5uf81g1-algo-1-sboaf | 2022-07-13 13:05:47,922 sagemaker-training-toolkit INFO     In

When you first run the SKLearn estimator, executing it may take some time as it has to download the scikit-learn container to the local docker environment. You will get the container logs in the output when the container completes the execution. The logs show that the container has successfully run the training script, and the `hello` message is also printed. But there is a lot more information available in the logs. So in the coming section, we will discuss that.

![sklearn-output-1](images/2022-07-07-sagemaker-script-mode/sklearn-output-1.png)

# Understanding SKLearn container output and environment varaibles
From the SKLearn estimator output, we can see that our `train_and_server.py` script is executed by the container with the following command.

```
/miniconda3/bin/python train_and_server.py
```

## Inspecting SageMaker SKLearn docker image
Since the container was executed in the local environment, we can also inspect the SageMaker SKLearn local image.

In [17]:
!docker images

REPOSITORY                                                            TAG             IMAGE ID       CREATED       SIZE
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn   1.0-1-cpu-py3   ff85d0034e62   5 weeks ago   3.63GB


Let's also inspect the docker image. Notice the multiple container environment variables and their default values.

In [18]:
#collapse-output
!docker inspect ff85d0034e62

[
    {
        "Id": "sha256:ff85d0034e624cce6f1114c32807437e2d5fd13b3593caeadb736db13bc7e2a8",
        "RepoTags": [
            "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.0-1-cpu-py3"
        ],
        "RepoDigests": [
            "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn@sha256:2ec1580756e6135455bcbcdff996c87bac72a0c8aa3bd924ff79372bd294f37a"
        ],
        "Parent": "",
        "Comment": "",
        "Created": "2022-06-02T23:29:51.803646532Z",
        "Container": "24e79034ec0d3a22e1ef5059f99c18e3a496fd49beb41206560f5018de6f07f2",
        "ContainerConfig": {
            "Hostname": "24e79034ec0d",
            "Domainname": "",
            "User": "",
            "AttachStdin": false,
            "AttachStdout": false,
            "AttachStderr": false,
            "ExposedPorts": {
                "8080/tcp": {}
            },
            "Tty": false,
            "OpenStdin": false,
            "StdinOnce": false,
    

## Pass hyperparameters to SKLearn estimator

Let's pass some dummy hyperparameters to the estimator and see how it affects the output.

In [19]:
#collapse-output
sk_estimator = SKLearn(
    entry_point=script_file,
    role=role,
    instance_count=1,
    instance_type='local',
    framework_version="1.0-1",
    hyperparameters={"dummy_param_1":"val1","dummy_param_2":"val2"},
)

sk_estimator.fit()

Creating vatky0ulv7-algo-1-79qnt ... 
Creating vatky0ulv7-algo-1-79qnt ... done
Attaching to vatky0ulv7-algo-1-79qnt
vatky0ulv7-algo-1-79qnt | 2022-07-13 13:05:50,867 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
vatky0ulv7-algo-1-79qnt | 2022-07-13 13:05:50,871 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
vatky0ulv7-algo-1-79qnt | 2022-07-13 13:05:50,880 sagemaker_sklearn_container.training INFO     Invoking user training script.
vatky0ulv7-algo-1-79qnt | 2022-07-13 13:05:51,053 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
vatky0ulv7-algo-1-79qnt | 2022-07-13 13:05:51,067 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
vatky0ulv7-algo-1-79qnt | 2022-07-13 13:05:51,083 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
vatky0ulv7-algo-1-79qnt | 2022-07-13 13:05:51,092 sagemaker-training-toolkit INFO     In

![sklearn-output-hyperparams](images/2022-07-07-sagemaker-script-mode/sklearn-output-hyperparams.png)

From the output we can see that our hyperparameters were passed to our training script as command line arguments. This is an important point and we will update our script using this information.


## SageMaker SKLearn container environment variables
Let's now discuss the essential environment variables we see in the output.

### SM_MODULE_DIR
```
SM_MODULE_DIR=s3://sagemaker-us-east-1-801598032724/sagemaker-scikit-learn-2022-07-13-13-05-48-675/source/sourcedir.tar.gz
```
`SM_MODULE_DIR` points to a location in the S3 bucket where SageMaker will automatically backup our source code for that particular run. SageMaker will create a separate folder in the default bucket for each new run. The default value is `s3://sagemaker-{aws-region}-{aws-id}/{training-job-name}/source/sourcedir.tar.gz`

### SM_MODEL_DIR
```
SM_MODEL_DIR=/opt/ml/model
```
`SM_MODEL_DIR` points to a directory located inside the container. When the training job finishes, the container and its file system will be deleted, except for the `/opt/ml/model` and `/opt/ml/output` directories. Use `/opt/ml/model` to save the trained model artifacts. These artifacts are uploaded to S3 for model hosting.

### SM_OUTPUT_DATA_DIR
```
SM_OUTPUT_DIR=/opt/ml/output
```
`SM_OUTPUT_DIR` points to a directory in the container to write output artifacts. Output artifacts may include checkpoints, graphs, and other files to save, not including model artifacts. These artifacts are compressed and uploaded to S3 to the same S3 prefix as the model artifacts.

### SM_CHANNELS
From the SKLearn estimator output
```
SM_CHANNELS='["testing","training"]'
```
A channel is a named input source that training algorithms can consume. You can partition your training data into different logical "channels" when you run training. Depending on your problem, some common channel ideas are: "training", "testing", "evaluation" or "images" and "labels". You can read more about the channels from SageMaker API reference [Channel](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_Channel.html)

### SM_CHANNEL_{channel_name}
```
SM_CHANNEL_TRAIN='/opt/ml/input/data/train'
SM_CHANNEL_TEST='/opt/ml/input/data/test'
```
Supposing that you have passed two input channels, 'train' and 'test', to the Scikit-learn estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:
* **`SM_CHANNEL_TRAIN`**: it points to the directory in the container that has the *train* channel data downloaded
* **`SM_CHANNEL_TEST`**: Same as above, but for the *test* channel

Note that the channel names `train` and `test` are the conventions. Still, you can use any name here, and the environment variables will be created accordingly. It is important to know that the SageMaker container automatically downloads the data from the provided input channels and makes them available in the respective local directories once it starts executing. The training script can then load the data from the local container directories.

There are more environment variables available, and you can read about them from [Environment variables](https://github.com/aws/sagemaker-training-toolkit/blob/master/ENVIRONMENT_VARIABLES.md)

# Pass input channel to SKLearn estimator

Now that we understand the SKLearn container environment more let's pass the training data channel to the estimator and see if the data becomes available inside the container directory. 

Update our script to list all the files in the `SM_CHANNEL_TRAIN` directory.

In [20]:
%%writefile $script_file
import argparse, os, sys

if __name__ == "__main__":
    print(" *** Hello from SageMaker script container *** ")

    training_dir = os.environ.get("SM_CHANNEL_TRAIN")
    dir_list = os.listdir(training_dir)

    print("training_dir files list: ", dir_list)


Overwriting ./datasets/2022-07-07-sagemaker-script-mode/src/train_and_serve.py


In [21]:
#collapse-output
sk_estimator = SKLearn(
    entry_point=script_file,
    role=role,
    instance_count=1,
    instance_type='local',
    framework_version="1.0-1",
    hyperparameters={"dummy_param_1":"val1","dummy_param_2":"val2"},
)

sk_estimator.fit({"train": f"file://{local_train_path}"})

Creating ajvs4neawf-algo-1-vcoo5 ... 
Creating ajvs4neawf-algo-1-vcoo5 ... done
Attaching to ajvs4neawf-algo-1-vcoo5
ajvs4neawf-algo-1-vcoo5 | 2022-07-13 13:05:53,961 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
ajvs4neawf-algo-1-vcoo5 | 2022-07-13 13:05:53,966 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
ajvs4neawf-algo-1-vcoo5 | 2022-07-13 13:05:53,975 sagemaker_sklearn_container.training INFO     Invoking user training script.
ajvs4neawf-algo-1-vcoo5 | 2022-07-13 13:05:54,165 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
ajvs4neawf-algo-1-vcoo5 | 2022-07-13 13:05:54,179 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
ajvs4neawf-algo-1-vcoo5 | 2022-07-13 13:05:54,193 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
ajvs4neawf-algo-1-vcoo5 | 2022-07-13 13:05:54,202 sagemaker-training-toolkit INFO     In

**[image here]**
From the output, we can see that `train.csv`, which was in our local environment, is now available inside the container on path `SM_CHANNEL_TRAIN=/opt/ml/input/data/train`. 

Let's also test the same with our training data on the S3 bucket.

In [22]:
#collapse-output
sk_estimator = SKLearn(
    entry_point=script_file,
    role=role,
    instance_count=1,
    instance_type='local',
    framework_version="1.0-1",
    hyperparameters={"dummy_param_1":"val1","dummy_param_2":"val2"},
)

sk_estimator.fit({"train": s3_train_uri})

Creating ej721wg4x0-algo-1-4b1nb ... 
Creating ej721wg4x0-algo-1-4b1nb ... done
Attaching to ej721wg4x0-algo-1-4b1nb
ej721wg4x0-algo-1-4b1nb | 2022-07-13 13:05:56,826 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
ej721wg4x0-algo-1-4b1nb | 2022-07-13 13:05:56,833 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
ej721wg4x0-algo-1-4b1nb | 2022-07-13 13:05:56,842 sagemaker_sklearn_container.training INFO     Invoking user training script.
ej721wg4x0-algo-1-4b1nb | 2022-07-13 13:05:57,027 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
ej721wg4x0-algo-1-4b1nb | 2022-07-13 13:05:57,041 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
ej721wg4x0-algo-1-4b1nb | 2022-07-13 13:05:57,054 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
ej721wg4x0-algo-1-4b1nb | 2022-07-13 13:05:57,063 sagemaker-training-toolkit INFO     In

Again the results are the same. SageMaker will download the data from the S3 bucket and make it available in the container. In the environment variables section we also learned that two directories are special `/opt/ml/model` and `/opt/ml/output`. Container environment variables `SM_MODEL_DIR` and `SM_OUTPUT_DATA_DIR` point to them, respectively. Whatever artifacts we put on them will be stored on the S3 bucket when the training job finishes. "SM_MODEL_DIR" is for trained models, and "SM_OUTPUT_DATA_DIR" is for other artifacts like logs, graphs, plots, resutls, etc. Let's update our training script and put some dummy data in these directories. Once the job is complete, we will verify the stored artifacts on the S3 bucket.

In [23]:
%%writefile $script_file
import argparse, os, sys

if __name__ == "__main__":
    print(" *** Hello from SageMaker script container *** ")

    # list files in SM_CHANNEL_TRAIN
    training_dir = os.environ.get("SM_CHANNEL_TRAIN")
    dir_list = os.listdir(training_dir)
    print("training_dir files list: ", dir_list)

    # write dummy model file to SM_MODEL_DIR
    sm_model_dir = os.environ.get("SM_MODEL_DIR")
    with open(f"{sm_model_dir}/dummy-model.txt", "w") as f:
        f.write("this is a dummy model")

    # write dummy artifact file to SM_OUTPUT_DATA_DIR
    sm_output_data_dir = os.environ.get("SM_OUTPUT_DATA_DIR")
    with open(f"{sm_output_data_dir}/dummy-output-data.txt", "w") as f:
        f.write("this is a dummy output data")

Overwriting ./datasets/2022-07-07-sagemaker-script-mode/src/train_and_serve.py


In [24]:
#collapse-output
sk_estimator = SKLearn(
    entry_point=script_file,
    role=role,
    instance_count=1,
    instance_type='local',
    framework_version="1.0-1",
    hyperparameters={"dummy_param_1":"val1","dummy_param_2":"val2"},
)

sk_estimator.fit({"train": s3_train_uri})

Creating 7kxyzpqlld-algo-1-w41kb ... 
Creating 7kxyzpqlld-algo-1-w41kb ... done
Attaching to 7kxyzpqlld-algo-1-w41kb
7kxyzpqlld-algo-1-w41kb | 2022-07-13 13:05:59,899 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
7kxyzpqlld-algo-1-w41kb | 2022-07-13 13:05:59,903 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
7kxyzpqlld-algo-1-w41kb | 2022-07-13 13:05:59,913 sagemaker_sklearn_container.training INFO     Invoking user training script.
7kxyzpqlld-algo-1-w41kb | 2022-07-13 13:06:00,115 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
7kxyzpqlld-algo-1-w41kb | 2022-07-13 13:06:00,129 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
7kxyzpqlld-algo-1-w41kb | 2022-07-13 13:06:00,142 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
7kxyzpqlld-algo-1-w41kb | 2022-07-13 13:06:00,151 sagemaker-training-toolkit INFO     In

Failed to delete: /tmp/tmpgsc_yr3n/algo-1-w41kb Please remove it manually.


===== Job Complete =====


Our training job is now complete. Let us check the S3 bucket to see if our dummy model and other artifacts are present.

First, we need the S3 URI for these artifacts. For our dummy model (from SM_MODEL_DIR), we can use our estimator object to get the URI. Let's call it `model_data`.

In [25]:
model_data = sk_estimator.model_data
model_data

's3://sagemaker-us-east-1-801598032724/sagemaker-scikit-learn-2022-07-13-13-05-57-790/model.tar.gz'

`model_data` from S3 will be downloaded to a local directory for verification. Let's create a local `tmp` to store these downloaded files.

In [26]:
local_tmp_path = local_path + "/tmp"
print(local_tmp_path)

# create the local '/tmp' directory
Path(local_tmp_path).mkdir(parents=True, exist_ok=True)

'./datasets/2022-07-07-sagemaker-script-mode/model'

We will use SageMaker `S3Downloader` object to download the model file.

In [27]:
from sagemaker.s3 import S3Downloader

S3Downloader.download(
    s3_uri=model_data, local_path=local_tmp_path, sagemaker_session=session
)

File is downloaded. Let's uncompress it to verify the model file.

In [28]:
!tar -xzvf $local_model_path/model.tar.gz -C $local_tmp_path

dummy-model.txt


Yes, the "dummy-model.txt" file is present. This tells us that SageMaker will automatically upload the files from the model directory (SM_MODEL_DIR) to the S3 bucket. Let's do the same for the output data directory (SM_OUTPUT_DATA_DIR). There is no direct way to get the S3 URI from the estimator object for the output data directory. But we can prepare it ourselves. So let's do that next.

In [ ]:
print("estimator.output_path: ", estimator.output_path)
print("estimator.latest_training_job.name: ", estimator.latest_training_job.name)

In [30]:
def get_s3_output_uri(estimator):
    return estimator.output_path + estimator.latest_training_job.name
    
get_s3_output_uri(sk_estimator)

's3://sagemaker-us-east-1-801598032724/sagemaker-scikit-learn-2022-07-13-13-05-57-790'

In [31]:
##
# S3 URI for output data artifacts
s3_output_uri = get_s3_output_uri(sk_estimator) + '/output.tar.gz'
s3_output_uri

's3://sagemaker-us-east-1-801598032724/sagemaker-scikit-learn-2022-07-13-13-05-57-790/output.tar.gz'

In [32]:
## 
# S3 URI for model artifact. We have already veirifed it.
s3_model_uri = get_s3_output_uri(sk_estimator) + '/model.tar.gz'
s3_model_uri

's3://sagemaker-us-east-1-801598032724/sagemaker-scikit-learn-2022-07-13-13-05-57-790/model.tar.gz'

In [33]:
##
# S3 URI for source code
s3_source_uri  get_s3_output_uri(sk_estimator) + '/source/sourcedir.tar.gz'
s3_source_uri

's3://sagemaker-us-east-1-801598032724/sagemaker-scikit-learn-2022-07-13-13-05-57-790/source/sourcedir.tar.gz'

Let's download these artifacts to our local '/tmp' directory for verification.

In [ ]:
!aws s3 cp $s3_output_uri $local_tmp_path
!aws s3 cp $s3_source_uri $local_tmp_path

In [ ]:
##
# extract the output data files from 'output.tar.gz'
!tar -xzvf $local_tmp_path/output.tar.gz -C $local_tmp_path

In [ ]:
##
# extract the source code files from 'sourcedir.tar.gz'
!tar -xzvf $local_tmp_path/sourcedir.tar.gz -C $local_tmp_path

# Summary till now
Let's summarize what we have learned till now.
* We can use SageMaker SKLearn local mode to test our code in a local environment
* SKLearn container executes our provided script with the command `/miniconda3/bin/python train_and_server.py`
* Hyperparameters passed to the container are passed to our script as command line arguments
* Data from input channels will be downloaded by the container and made available for our script to load and process
* '/opt/ml/model' and '/opt/ml/output' directories are special. Anything stored on them will be automatically backed up on the S3 bucket when the job finishes. These directories are defined in the container environment variables 'SM_MODEL_DIR' and 'SM_OUTPUT_DATA_DIR', respectively. SM_MODEL_DIR should be used to write model artifacts. SM_OUTPUT_DATA_DIR should be used to write any other supporting artifact.

Let's use this knowledge to update our script to train a RandomForrestClassifier on the Iris flower dataset.

In [ ]:
# cleanup /tmp directory
!rm -r $local_tmp_path/*

In [35]:
%%writefile $script_file


import argparse, os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import joblib

if __name__ == "__main__":

    # Pass in environment variables and hyperparameters
    parser = argparse.ArgumentParser()

    # Hyperparameters
    parser.add_argument("--estimators", type=int, default=15)

    # sm_model_dir: model artifacts stored here after training
    parser.add_argument("--sm-model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--sm-channel-train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--sm-channel-test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--sm-output-data-dir", type=str, default=os.environ.get("SM_OUTPUT_DATA_DIR"))
    
    args, _ = parser.parse_known_args()
    
    print("command line arguments: ", args)
    
    estimators = args.estimators
    sm_model_dir = args.sm_model_dir
    training_dir = args.sm_channel_train
    testing_dir = args.sm_channel_test
    output_data_dir = args.sm_output_data_dir
    
    print(f"training_dir: {training_dir}") # print training_dir path
    print(f"training_dir files list: {os.listdir(training_dir)}") # print training_dir files list
    print(f"testing_dir: {testing_dir}") # print testing_dir path
    print(f"testing_dir files list: {os.listdir(testing_dir)}") # print testing_dir files list
    print(f"sm_model_dir: {sm_model_dir}")
    print(f"output_data_dir: {output_data_dir}")
    
    
    # Read in data
    df_train = pd.read_csv(training_dir + "/train.csv", sep=",")
    df_test = pd.read_csv(testing_dir + "/test.csv", sep=",")

    # Preprocess data
    X_train = df_train.drop(["class", "class_cat"], axis=1)
    y_train = df_train["class_cat"]
    X_test = df_test.drop(["class", "class_cat"], axis=1)
    y_test = df_test["class_cat"]
    
    print(f"X_train.shape: {X_train.shape}")
    print(f"y_train.shape: {y_train.shape}")
    print(f"X_train.shape: {X_test.shape}")
    print(f"y_train.shape: {y_test.shape}")
    
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    # Build model
    regressor = RandomForestClassifier(n_estimators=estimators)
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)

    # Save the model
    joblib.dump(regressor, sm_model_dir+"/model.joblib")
    
    # Save the results
    pd.DataFrame(y_pred).to_csv(output_data_dir+"/y_pred.csv")
    
    # print sm_model_dir info
    print(f"sm_model_dir: {sm_model_dir}") # print sm_model_dir path
    print(f"sm_model_dir files list: {os.listdir(sm_model_dir)}") # print sm_model_dir files list

Overwriting ./datasets/2022-07-07-sagemaker-script-mode/src/train_and_serve.py


In [36]:
!chmod +x $script_file

In [37]:
# !python3 --version

In [38]:
# !conda env list

In [39]:
# clean model output directory
!rm -r $local_model_path/*

In [40]:
!python3 $script_file \
    --sm-model-dir $local_model_path \
    --sm-channel-train $local_train_path \
    --sm-channel-test $local_test_path \
    --sm-output-data-dir $local_output_data_path \
    --estimators 10

command line arguments:  Namespace(estimators=10, sm_channel_test='./datasets/2022-07-07-sagemaker-script-mode/test', sm_channel_train='./datasets/2022-07-07-sagemaker-script-mode/train', sm_model_dir='./datasets/2022-07-07-sagemaker-script-mode/model', sm_output_data_dir='./datasets/2022-07-07-sagemaker-script-mode/output/data')
training_dir: ./datasets/2022-07-07-sagemaker-script-mode/train
training_dir files list: ['train.csv', '.ipynb_checkpoints']
testing_dir: ./datasets/2022-07-07-sagemaker-script-mode/test
testing_dir files list: ['.ipynb_checkpoints', 'test.csv']
sm_model_dir: ./datasets/2022-07-07-sagemaker-script-mode/model
output_data_dir: ./datasets/2022-07-07-sagemaker-script-mode/output/data
X_train.shape: (120, 4)
y_train.shape: (120,)
X_train.shape: (30, 4)
y_train.shape: (30,)
sm_model_dir: ./datasets/2022-07-07-sagemaker-script-mode/model
sm_model_dir files list: ['.ipynb_checkpoints', 'model.joblib']


In [41]:
local_model_path

'./datasets/2022-07-07-sagemaker-script-mode/model'

In [42]:
# we have tested our script locally. so let's test this with SageMaker SKlean container

In [43]:
sk_estimator = SKLearn(
    entry_point=script_file,
    role=role,
    instance_count=1,
    instance_type='local',
    framework_version="1.0-1",
    hyperparameters={"estimators":10},
)

# Train the estimator
sk_estimator.fit({"train": s3_train_uri, "test": s3_test_uri})

Creating 0b3yb3qvjt-algo-1-n90bz ... 
Creating 0b3yb3qvjt-algo-1-n90bz ... done
Attaching to 0b3yb3qvjt-algo-1-n90bz
0b3yb3qvjt-algo-1-n90bz | 2022-07-13 13:06:06,304 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
0b3yb3qvjt-algo-1-n90bz | 2022-07-13 13:06:06,308 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
0b3yb3qvjt-algo-1-n90bz | 2022-07-13 13:06:06,317 sagemaker_sklearn_container.training INFO     Invoking user training script.
0b3yb3qvjt-algo-1-n90bz | 2022-07-13 13:06:06,498 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
0b3yb3qvjt-algo-1-n90bz | 2022-07-13 13:06:06,512 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
0b3yb3qvjt-algo-1-n90bz | 2022-07-13 13:06:06,525 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
0b3yb3qvjt-algo-1-n90bz | 2022-07-13 13:06:06,534 sagemaker-training-toolkit INFO     In

Failed to delete: /tmp/tmp5kv1g_u7/algo-1-n90bz Please remove it manually.


===== Job Complete =====


In [44]:
sk_estimator.model_data

's3://sagemaker-us-east-1-801598032724/sagemaker-scikit-learn-2022-07-13-13-06-03-671/model.tar.gz'

In [45]:
s3_output_uri = get_s3_output_uri(sk_estimator) + '/output.tar.gz'
s3_output_uri

's3://sagemaker-us-east-1-801598032724/sagemaker-scikit-learn-2022-07-13-13-06-03-671/output.tar.gz'

In [46]:
local_output_data_path

'./datasets/2022-07-07-sagemaker-script-mode/output/data'

In [47]:
print(local_output_data_path)

!rm -r $local_output_data_path/*

./datasets/2022-07-07-sagemaker-script-mode/output/data


In [48]:
!aws s3 cp $s3_output_uri $local_output_data_path

download: s3://sagemaker-us-east-1-801598032724/sagemaker-scikit-learn-2022-07-13-13-06-03-671/output.tar.gz to datasets/2022-07-07-sagemaker-script-mode/output/data/output.tar.gz


In [49]:
!tar -xzvf $local_output_data_path/output.tar.gz -C $local_output_data_path

data/
data/y_pred.csv
success


In [50]:
##### requirements and custom_library

In [51]:
# task we need to generate confusion matrix using seaborn library and store it in output data driectory
# create 

In [52]:
# create a custom library to save a seaborn CM

In [53]:
custom_library_path = local_path+"/my_custom_library"
custom_library_file = custom_library_path+"/seaborn_confusion_matrix.py"

print(f"custom_library_path: {custom_library_path}")
print(f"custom_library_file: {custom_library_file}")

custom_library_path: ./datasets/2022-07-07-sagemaker-script-mode/my_custom_library
custom_library_file: ./datasets/2022-07-07-sagemaker-script-mode/my_custom_library/seaborn_confusion_matrix.py


In [54]:
# create custom library folder
Path(custom_library_path).mkdir(parents=True, exist_ok=True)

In [55]:
%%writefile $custom_library_file

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import argparse, os

def save_confusion_matrix(cf_matrix, path="./"):
    sns_plot = sns.heatmap(cf_matrix, annot=True)
    sns_plot.figure.savefig(path+"/output_cm.png")
    
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--path", type=str, default="./")
    args, _ = parser.parse_known_args()
    path = args.path
    
    dummy_cm = np.array([[23,  5],[ 3, 30]])
    save_confusion_matrix(dummy_cm, path)

Overwriting ./datasets/2022-07-07-sagemaker-script-mode/my_custom_library/seaborn_confusion_matrix.py


In [56]:
%%writefile $custom_library_path/__init__.py

from .seaborn_confusion_matrix import *

Overwriting ./datasets/2022-07-07-sagemaker-script-mode/my_custom_library/__init__.py


In [57]:
!python3 $custom_library_file --path $local_output_data_path

In [58]:
%%writefile $script_file


import argparse, os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import joblib

from my_custom_library import save_confusion_matrix

if __name__ == "__main__":

    # Pass in environment variables and hyperparameters
    parser = argparse.ArgumentParser()

    # Hyperparameters
    parser.add_argument("--estimators", type=int, default=15)

    # sm_model_dir: model artifacts stored here after training
    parser.add_argument("--sm-model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--sm-channel-train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--sm-channel-test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--sm-output-data-dir", type=str, default=os.environ.get("SM_OUTPUT_DATA_DIR"))
    
    args, _ = parser.parse_known_args()
    
    print("command line arguments: ", args)
    
    estimators = args.estimators
    sm_model_dir = args.sm_model_dir
    training_dir = args.sm_channel_train
    testing_dir = args.sm_channel_test
    output_data_dir = args.sm_output_data_dir
    
    print(f"training_dir: {training_dir}") # print training_dir path
    print(f"training_dir files list: {os.listdir(training_dir)}") # print training_dir files list
    print(f"testing_dir: {testing_dir}") # print testing_dir path
    print(f"testing_dir files list: {os.listdir(testing_dir)}") # print testing_dir files list
    print(f"sm_model_dir: {sm_model_dir}")
    print(f"output_data_dir: {output_data_dir}")
    
    
    # Read in data
    df_train = pd.read_csv(training_dir + "/train.csv", sep=",")
    df_test = pd.read_csv(testing_dir + "/test.csv", sep=",")

    # Preprocess data
    X_train = df_train.drop(["class", "class_cat"], axis=1)
    y_train = df_train["class_cat"]
    X_test = df_test.drop(["class", "class_cat"], axis=1)
    y_test = df_test["class_cat"]
    
    print(f"X_train.shape: {X_train.shape}")
    print(f"y_train.shape: {y_train.shape}")
    print(f"X_train.shape: {X_test.shape}")
    print(f"y_train.shape: {y_test.shape}")
    
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    # Build model
    regressor = RandomForestClassifier(n_estimators=estimators)
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)

    # Save the model
    joblib.dump(regressor, sm_model_dir+"/model.joblib")
    
    # Save the results
    pd.DataFrame(y_pred).to_csv(output_data_dir+"/y_pred.csv")
    
    # save the confusion matrix
    cf_matrix = confusion_matrix(y_test, y_pred)
    save_confusion_matrix(cf_matrix, output_data_dir)
    
    # print sm_model_dir info
    print(f"sm_model_dir: {sm_model_dir}") # print sm_model_dir path
    print(f"sm_model_dir files list: {os.listdir(sm_model_dir)}") # print sm_model_dir files list
    
    # print output_data_dir info
    print(f"output_data_dir: {output_data_dir}") # print sm_model_dir path
    print(f"output_data_dir files list: {os.listdir(output_data_dir)}") # print sm_model_dir files list

Overwriting ./datasets/2022-07-07-sagemaker-script-mode/src/train_and_serve.py


In [59]:
script_path

'./datasets/2022-07-07-sagemaker-script-mode/src'

In [64]:
%%writefile $script_path/requirements.txt

matplotlib==3.5.2
seaborn==0.11.2

Overwriting ./datasets/2022-07-07-sagemaker-script-mode/src/requirements.txt


In [65]:
sk_estimator = SKLearn(
    entry_point=script_file_name,
    source_dir=script_path,
    dependencies=[custom_library_path],
    role=role,
    instance_count=1,
    instance_type='local',
    framework_version="1.0-1",
    hyperparameters={"estimators":10},
)

# Train the estimator
sk_estimator.fit({"train": s3_train_uri, "test": s3_test_uri})

Creating 96mwnip853-algo-1-gcn8f ... 
Creating 96mwnip853-algo-1-gcn8f ... done
Attaching to 96mwnip853-algo-1-gcn8f
96mwnip853-algo-1-gcn8f | 2022-07-13 13:07:46,389 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
96mwnip853-algo-1-gcn8f | 2022-07-13 13:07:46,393 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
96mwnip853-algo-1-gcn8f | 2022-07-13 13:07:46,403 sagemaker_sklearn_container.training INFO     Invoking user training script.
96mwnip853-algo-1-gcn8f | 2022-07-13 13:07:46,574 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
96mwnip853-algo-1-gcn8f | /miniconda3/bin/python -m pip install -r requirements.txt
96mwnip853-algo-1-gcn8f | Collecting matplotlib==3.5.2
96mwnip853-algo-1-gcn8f |   Downloading matplotlib-3.5.2-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 98.0 MB/s eta 0:00:0000:0100:01:--:-

Failed to delete: /tmp/tmp58lfmx49/algo-1-gcn8f Please remove it manually.


===== Job Complete =====


At this point we have a trained model. Can we deploy this model already?

If I try to deploy this model using command
```
sk_predictor = sk_estimator.deploy(
    initial_instance_count=1,
    instance_type='local'
)
```
i will get exception messages 
```
[2022-07-09 06:15:45 +0000] [31] [ERROR] Error handling request /ping
Traceback (most recent call last):
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_containers/_functions.py", line 93, in wrapper
    return fn(*args, **kwargs)
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_sklearn_container/serving.py", line 43, in default_model_fn
    return transformer.default_model_fn(model_dir)
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_containers/_transformer.py", line 35, in default_model_fn
    raise NotImplementedError(
NotImplementedError: 
Please provide a model_fn implementation.
See documentation for model_fn at https://github.com/aws/sagemaker-python-sdk
```
this is because Before a model can be served, it must be loaded. The SageMaker Scikit-learn model server loads your model by invoking a model_fn function that you must provide in your script. The model_fn should have the following signature:

def model_fn(model_dir)


In [ ]:
%%writefile $script_file

import argparse, os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import joblib

if __name__ == "__main__":

    # Pass in environment variables and hyperparameters
    parser = argparse.ArgumentParser()

    # Hyperparameters
    parser.add_argument("--estimators", type=int, default=15)

    # sm_model_dir: model artifacts stored here after training
    parser.add_argument("--sm-model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))

    args, _ = parser.parse_known_args()
    estimators = args.estimators
    sm_model_dir = args.sm_model_dir
    training_dir = args.train
    
    # print training_dir info
    print(f"training_dir: {training_dir}") # print training_dir path
    print(f"training_dir files list: {os.listdir(training_dir)}") # print training_dir files list
    
    # Read in data
    df = pd.read_csv(training_dir + "/train.csv", sep=",")

    # Preprocess data
    X = df.drop(["class", "class_cat"], axis=1)
    y = df["class_cat"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    # Build model
    regressor = RandomForestRegressor(n_estimators=estimators)
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)

    # Save model
    joblib.dump(regressor, os.path.join(sm_model_dir, "model.joblib"))
    
    # print sm_model_dir info
    print(f"sm_model_dir: {sm_model_dir}") # print sm_model_dir path
    print(f"sm_model_dir files list: {os.listdir(sm_model_dir)}") # print sm_model_dir files list

    
# Model serving
"""
Deserialize fitted model
"""
def model_fn(model_dir):
    print(f"model_fn model_dir: {model_dir}")
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    return model

"""
predict_fn
    input_data: returned array from input_fn above
    model (sklearn model) returned model loaded from model_fn above
"""
def predict_fn(input_data, model):
    return model.predict(input_data)

In [ ]:
sk_estimator = SKLearn(
    entry_point=script_file,
    role=role,
    instance_count=1,
    instance_type='local',
    framework_version="1.0-1",
    hyperparameters={"estimators":10},
)

# Train the estimator
sk_estimator.fit({"train": s3_train_uri})

In [ ]:
sk_predictor = sk_estimator.deploy(
    initial_instance_count=1,
    instance_type='local'
)

In [ ]:
request = [[9.0, 3571, 1976, 0.525]]

response  = sk_predictor.predict(request)
response = int(response[0])
response

In [ ]:
print("Predicted class category {} ({})".format(response, categories_map[response]))

In [ ]:
# sk_predictor.delete_model()

In [ ]:
sk_predictor.delete_endpoint()

Note:
* if you are already running a local endpoint then you will not be able to create a new one. if you have lost the kernel session then you can delete the running endpoint from terminal using kill command.

In [ ]:
# convert the inputs and outputs
# input in json and output in json

In [ ]:
%%writefile $script_file

import argparse, os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler # do i need this?
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import joblib
import json

if __name__ == "__main__":

    # Pass in environment variables and hyperparameters
    parser = argparse.ArgumentParser()

    # Hyperparameters
    parser.add_argument("--estimators", type=int, default=15)

    # sm_model_dir: model artifacts stored here after training
    parser.add_argument("--sm-model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))

    args, _ = parser.parse_known_args()
    estimators = args.estimators
    sm_model_dir = args.sm_model_dir
    training_dir = args.train
    
    # print training_dir info
    print(f"training_dir: {training_dir}") # print training_dir path
    print(f"training_dir files list: {os.listdir(training_dir)}") # print training_dir files list
    
    # Read in data
    df = pd.read_csv(training_dir + "/train.csv", sep=",")

    # Preprocess data
    X = df.drop(["class", "class_cat"], axis=1)
    y = df["class_cat"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    # Build model
    regressor = RandomForestRegressor(n_estimators=estimators)
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)

    # Save model
    joblib.dump(regressor, os.path.join(sm_model_dir, "model.joblib"))
    
    # print sm_model_dir info
    print(f"sm_model_dir: {sm_model_dir}") # print sm_model_dir path
    print(f"sm_model_dir files list: {os.listdir(sm_model_dir)}") # print sm_model_dir files list

    
# Model serving
"""
Deserialize fitted model
"""
def model_fn(model_dir):
    print(f"model_fn model_dir: {model_dir}")
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    return model

"""
predict_fn
    input_data: returned array from input_fn above
    model (sklearn model) returned model loaded from model_fn above
"""
def predict_fn(input_data, model):
    return model.predict(input_data)


"""
input_fn
    request_body: The body of the request sent to the model.
    request_content_type: (string) specifies the format/variable type of the request
"""
def input_fn(request_body, request_content_type):
    if request_content_type == "application/json":
        request_body = json.loads(request_body)
        inpVar = request_body["Input"]
        return inpVar
    else:
        raise ValueError("This model only supports application/json input")


"""
output_fn
    prediction: the returned value from predict_fn above
    content_type: the content type the endpoint expects to be returned. Ex: JSON, string
"""
def output_fn(prediction, content_type):
    res = int(prediction[0])
    respJSON = {"Output": res}
    return respJSON

In [ ]:
sk_estimator = SKLearn(
    entry_point=script_file,
    role=role,
    instance_count=1,
    instance_type='local',
    framework_version="1.0-1",
    hyperparameters={"estimators":10},
)

# Train the estimator
sk_estimator.fit({"train": s3_train_uri})

sk_predictor = sk_estimator.deploy(
    initial_instance_count=1,
    instance_type='local'
)

In [ ]:
sk_endpoint_name = sk_predictor.endpoint_name
sk_endpoint_name

In [ ]:
import json

client = session_local.sagemaker_runtime_client

request_body = {"Input": [[9.0, 3571, 1976, 0.525]]}
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)

response = client.invoke_endpoint(
    EndpointName=sk_endpoint_name, ContentType="application/json", Body=payload
)

result = json.loads(response["Body"].read().decode())["Output"]

In [ ]:
print("Predicted class category {} ({})".format(result, categories_map[result]))